# Overview

**Objectives:**

*   Establish a conversation with the poet Rubén Darío using RAG through his autobiography.

**Input:** Human question + most relevant sentences in the book "Autobiografía"

**Models:**

*   `all-MiniLM-L6-v2` for embeddings
*   `gpt-3.5-turbo` for Question Answering

**Expected Output:** Text resembling what the poet would answer based on his memoirs contained in the book *Autobiografía*


References:

*   [Autobiografía](https://www.gutenberg.org/ebooks/52050) by Rubén Darío
*   [LangChain: Chat with your data](https://www.deeplearning.ai/short-courses/langchain-chat-with-your-data/) Course

Follow the code below or [go straight to the results](#conclusion)

# Basic Imports

In [ ]:
!pip install langchain

In [ ]:
!pip install openai

In [ ]:
!pip3 install python-dotenv

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [ ]:
os.environ["OPENAI_API_KEY"] = "Insert your API key here"

# Clean and prepare dataset

In [ ]:
files_gutemberg = {
    "Autobiografia": { "file_name": "autobiografia.txt", "url": "https://www.gutenberg.org/cache/epub/52050/pg52050.txt", "encoding": "utf-8" }
    }

In [ ]:
path = tf.keras.utils.get_file("Autobiografia", "https://www.gutenberg.org/cache/epub/52050/pg52050.txt")
text = open(path, "rb").read().decode(encoding="utf-8")
print(f"Autobiografia")
print(f"Length of text: {len(text):,} characters")
print()


265017/265017 [==============================] - 0s 1us/step
Autobiografia
Length of text: 258,874 characters



In [ ]:
text

In [ ]:
# Remove anything before the first sentence of the book.

In [ ]:
substring = "(LA VITA DE BENVENUTO DE\r\nM.º CELLINI, FLORENTINO)."

if substring in text:
    print("Substring found!")
else:
    print("Substring not found.")

Substring found!


In [ ]:
# Split the string based on the sequence
split_result = text.split(substring, 1) # splits just 1 time

# Check if the sequence was found in the string
if len(split_result) > 1:
    result_string = split_result[1].lstrip()
else:
    result_string = text

In [ ]:
# Remove anything after the last sentence of the book.

In [ ]:
substring = "Buenos Aires, 11 de Septiembre.--5 de Octubre de 1912."

if substring in text:
    print("Substring found!")
else:
    print("Substring not found.")

Substring found!


In [ ]:
# Remove anything after the last sentence of the book.
sequence = "Buenos Aires, 11 de Septiembre.--5 de Octubre de 1912."

# Split the string based on the sequence
split_result = result_string.split(sequence, 1) # splits just one time

# Check if the sequence was found in the string
if len(split_result) > 1:
    result_string = split_result[0]
else:
    result_string = text



In [ ]:
# Remove utf-8 special characters

In [ ]:
import re

# Sample text with Spanish characters
text = "«¿Cuándo sale el primer vapor?»\r\n«Pasado mañana». «¡Pues me embarcaré pasado mañana!».\r\n\r\nDos días después iba yo navegando con rumbo a Europa."

# Replace all characters that are not alphabets, numbers, spaces, newlines, common punctuation,
# or Spanish special characters with spaces
cleaned_text = re.sub(r"[^a-zA-Z0-9 \n.,!?;'\"-«»áéíóúüÁÉÍÓÚÜñÑ]", " ", text)

print(cleaned_text)


« Cuándo sale el primer vapor?» 
«Pasado mañana». «¡Pues me embarcaré pasado mañana!». 
 
Dos días después iba yo navegando con rumbo a Europa.


In [ ]:
# Replace all characters that are not alphabets, numbers, spaces, newlines, common punctuation,
# or Spanish special characters with spaces
cleaned_text = re.sub(r"[^a-zA-Z0-9 \n.,!?;'\"-«»áéíóúüÁÉÍÓÚÜñÑ]", " ", result_string)

print(cleaned_text)

In [ ]:
type(cleaned_text)

str

## Split sentences

In [ ]:
# Remove UTF-8 newline characters
text_without_newlines = cleaned_text.replace('\n', ' ')

# Remove double spaces
text_without_newlines = text_without_newlines.replace('  ', ' ')

# Split with period as separator and join if result < 30 characters
splits = re.split(r'(?<=[.])\s+', text_without_newlines)
sentences = []
buffer = ""

for split in splits:
    buffer += split
    if len(buffer) >= 30:
        sentences.append(buffer)
        buffer = ""
    else:
        buffer += ' '

# Check if there's any remaining content in the buffer
if buffer:
    sentences.append(buffer)


In [ ]:
import random
random.sample(sentences, 10)

['En la puerta de la calle, no lejos de mí, estaba la gente de la tertulia habitual.',
 'Llegué a la capital. Al cochero que me preguntó a qué hotel iba, le contesté sencillamente: «Al mejor».',
 'Con Domingo Martinto y Francisco Soto y Calvo, ambos cuñados de Obligado, ambos poetas y personas de distinción y afabilidad.',
 'Cuando llegué a París, la contrariedad del ministro Medina al saber que iba yo a sustituirle en su puesto diplomático de España--pues él era representante de Nicaragua en cuatro o cinco países de Europa--se exteriorizó con tal despecho, que me juró aquel provecto caballero no volver a poner los pies en España.',
 'XXX   A mi llegada a Nicaragua, permanecí algunos días en la ciudad de León.',
 'Su mujer no le iba en zaga como conversadora lista y pronta para la «ripposta»; y pude presenciar, en una de las comidas a que asistiera en el opulento palacio de la Huerta, en la Guindalera, a una justa de ingenio en que tomaban parte Cánovas, Joaquina, Castelar y el general

In [ ]:
# Find an individual sentence from a substring to validate length of split.

strings_list = sentences
substring = "Fulgencio Mayorca, en Panamá"

for value in strings_list:
    if substring in value:
        resultado = value
        break

print(resultado)


Fulgencio Mayorca, en Panamá. En el puerto de Colón tomamos pasaje en un vapor español de la compañía Trasatlántica, si mal no recuerdo el _León XIII_; y salimos con rumbo a Santander.


In [ ]:
len("Y me extendí sobre el particular.")

33

In [ ]:
type(sentences)

list

In [ ]:
len(sentences)

1631

## Calculate cost of embedding

In [ ]:
# Find all sequences of non-space characters and count them
word_count = len(re.findall(r'\S+', cleaned_text))

print(word_count)


37993


In [ ]:
# 100 tokens ~= 75 words
# https://help.openai.com/en/articles/4936856-what-are-tokens-and-how-to-count-them

aprox_tokens = (word_count*100) / 75

print(aprox_tokens)

50657.333333333336


In [ ]:
# GPT4 8K context is
# $0.03 / 1K tokens (input)
# $0.06 / 1K tokens (output)
# We would use GPT3.5 though

aprox_price = (aprox_tokens/1000) * 0.03

print(f"Approximate price: ${aprox_price:.2f}")

Approximate price: $1.52


# Vectorstores and embeddings

Embeddings capture the semantic information of the sentences and enable us to perform similarity calculations between them.

Dot product normalized by lenght of vector is cosine similarity--the higher the dot product result, the most similar two sentences are.

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [ ]:
!pip install chromadb

In [ ]:
from langchain.vectorstores import Chroma
from langchain.chains import VectorDBQA

In [ ]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


## Embed sentences and save in Gdrive (first time only)

Uncomment the following cells to create embeddings from the split sentences and store the resulting vectors in Gdrive.

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
# !ls /content/gdrive

In [ ]:
# import os
# DATADIR = os.path.join('/content/gdrive', Insert your folders here.)

In [ ]:
# DATADIR

In [ ]:
# persist_directory = DATADIR

In [ ]:
# # Create Document class as expected by Langchain
# # https://api.python.langchain.com/en/latest/schema/langchain.schema.document.Document.html#langchain.schema.document.Document

# from typing import Literal, Optional

# class Document:
#     def __init__(self, page_content: str, metadata: Optional[dict] = None, type: Literal['Document'] = 'Document'):
#         self.page_content = page_content
#         self.metadata = metadata
#         self.type = type

# # Sample list of strings
# string_list = sentences

# # Convert the list of strings to a list of Document objects
# document_list = [Document(page_content=s) for s in string_list]

# # Verify
# for doc in document_list:
#     print(doc.page_content)

`Chroma.from_documents` - Create a Chroma vectorstore from a list of documents.

If a persist_directory is specified, the collection will be persisted there. Otherwise, the data will be ephemeral in-memory.

In [ ]:
# # Create embeddings

# vectordb = Chroma.from_documents(
#     documents = document_list,
#     embedding = embedding,
#     persist_directory = persist_directory
# )

In [ ]:
# vectordb.persist() # this saves the vectorstore to the specified directory

## Upload embeddings

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
persist_directory = '/content/gdrive/My Drive/Insert the path to your embeddings here.'

In [ ]:
# vectordb = None # delete the instance (if any) to load it again below

In [ ]:
from chromadb.utils import embedding_functions

In [ ]:
embedding = OpenAIEmbeddings(openai_api_key = 'Insert your openai API key here.')

In [ ]:
vectordb = Chroma(persist_directory = persist_directory, embedding_function=embedding) # this line calls the vectordb from the directory

# Solved error:
# https://community.openai.com/t/load-embedding-from-disk-langchain-chroma-db/290297/2

In [ ]:
vectordb.get()

Other useful references:

*   By default, Chroma uses the Sentence Transformer all-MiniLM-L6-v2 model to create embeddings, read [here](https://docs.trychroma.com/embeddings)



## Test embeddings

In [ ]:
len(embedding.embed_documents(["EMBED THIS"])[0]) # Just to check that embeddings are working.

1536

In [ ]:
print(vectordb._collection.count())

# This number matches the number of sentences/documents

1631


# Similarity search

In [ ]:
question = '¿Me puede dar un ejemplo de lo que disfruta hacer?'

In [ ]:
docs = vectordb.similarity_search(question, k=3)

In [ ]:
len(docs)

3

In [ ]:
docs[0].page_content

'Otras cuantas aventuras de este género me acontecieron, pues en esa época yo hacía vida de café, con compañeros de existencia idéntica, y derrochaba mi juventud, sin economizar los medios de ponerla a prueba.'

# Retrieval

In [ ]:
texts = [
    """Más adelante, me interesé mucho por la obra de Victor Hugo, que tendría una influencia determinante en mi labor poética.""",
    """Este texto no tiene nada que ver con el autor ni con su autobiografía.""",
    """Mi religiosidad se aparta de la ortodoxia católica para buscar refugio en la religiosidad sincrética propia del fin de siglo, en la que se entremezclan influencias orientales y corrientes ocultistas."""
]

In [ ]:
smalldb = Chroma.from_texts(texts, embedding=embedding)

In [ ]:
question = "¿Qué opinas de la religión?"

In [ ]:
answers = smalldb.similarity_search(question, k=2)

In [ ]:
answers[0].page_content

'Mi religiosidad se aparta de la ortodoxia católica para buscar refugio en la religiosidad sincrética propia del fin de siglo, en la que se entremezclan influencias orientales y corrientes ocultistas.'

# Question Answering

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
llm = ChatOpenAI(model_name = 'gpt-3.5-turbo', temperature = 0)

## Retrieval QA chain

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = vectordb.as_retriever()
)

In [ ]:
question = "¿Cuál es tu primer recuerdo?"

In [ ]:
result = qa_chain({"query": question})

In [ ]:
result["result"]

'No tengo recuerdos personales ya que soy una inteligencia artificial y no tengo experiencias pasadas.'

## Prompt

In [ ]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Usa el siguiente contexto para contestar la pregunta.
Responde con el mismo estilo literario presente en el documento.
La interacción es parte de una conversación con un humano.
Si no sabes la respuesta, haz un comentario que ayude a continuar la conversación.
Al final, haz una pregunta sobre el mismo tema.
{context}
Question: {question}
Friendly answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [ ]:
# Run chain

llm = ChatOpenAI(model_name = 'gpt-3.5-turbo', temperature = 1)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [ ]:
question = "¿Qué representa la poesía para usted?"

In [ ]:
result = qa_chain({"query": question})

In [ ]:
result["result"]

'La poesía, para mí, es como el susurro del viento entre las hojas de los árboles, es la melodía de los pájaros al amanecer y el aroma embriagador de las rosas en plena floración. Es el lenguaje de los sentimientos profundos y de las emociones indescriptibles. Representa una ventana hacia lo inefable, una forma de expresar lo inexpresable. La poesía es un bálsamo para el alma, un refugio en medio del caos, un remanso de paz en el agitado mar de la vida. ¿Y para usted, qué representa la poesía?'

In [ ]:
result["source_documents"][0] # This shows the document where the answer is coming from.

Document(page_content='Poética, sí, para quien sepa comprender el vaho de arte que flota sobre ese inconmesurable océano de tierra, sobre todo en los crepúsculos vespertinos y en los amaneceres.')

In [ ]:
question = "¿Qué representa la poesía para usted?"
result = qa_chain({"query": question})
result["result"]

'La poesía representa para mí un puente que conecta el mundo tangible con lo intangible, un medio para expresar emociones y pensamientos de forma sublime y trascendental. Es el lenguaje de la belleza, el vaho de arte que flota en el aire, y que nos transporta a dimensiones más allá de lo cotidiano. Como el poeta y su ilusión, la poesía encarna la esperanza, la tristeza, el amor y la melancolía, revelando así la complejidad y la vulnerabilidad de la condición humana. Su poder radica en su capacidad de evocar imágenes y emociones con palabras cuidadosamente tejidas, y sumergirnos en un mundo de significado profundo y revelaciones ocultas. Para mí, la poesía es un refugio sagrado, un medio de conexión íntima con el alma y una forma de darle sentido a la realidad. ¿Cuál es tu poema favorito y por qué te impacta tanto?'

In [ ]:
result["source_documents"][0] # This shows the document where the answer is coming from.

Document(page_content='Poética, sí, para quien sepa comprender el vaho de arte que flota sobre ese inconmesurable océano de tierra, sobre todo en los crepúsculos vespertinos y en los amaneceres.')

# Chat

## Memory

In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

## Conversational retrieval chain

In [ ]:
from langchain.chains import ConversationalRetrievalChain
retriever=vectordb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

In [ ]:
question = "¿Quién es tu mejor amigo?"
result = qa_chain({"query": question})
result["result"]

'Mi mejor amigo es una persona que ha estado a mi lado en todas las etapas de mi vida, alguien a quien puedo confiar mis pensamientos más profundos y quien siempre me apoya en mis decisiones y sueños. Es una persona alegre, vibrante y con una gran pasión por la vida. Juntos hemos compartido risas, aventuras y también momentos difíciles. Aunque su nombre no es conocido por muchos, para mí es el mejor amigo que podría tener.'

In [ ]:
question = "¿Cómo se llama su amigo?"
result = qa_chain({"query": question})
result["result"]

'Mi amigo se llama Charles E.'

In [ ]:
result["source_documents"][0] # This shows the document where the answer is coming from.

Document(page_content='Había allí un secretario. Lo llamó.')

<a name="conclusion"></a>
# Conclusion

🥳 Success! By using RAG, we can "talk" to a poet through their work!

By embedding and retrieving relevant documents--the sentences in the book "Autobiografía" in this case--, we can obtain answers from the LLM based on the content present in the book.
As we can see in the example below, halucinations can somewhat be offset by changing the temperature. Example: When asked about one of the poet's most famous poems with `temperature = 0.7`, the LLM attributed the poem to a different author!🤯 By changing to `temperature = 1`, this fact was corrected.

Advantages of using **RAG vs the typical LLM chat platforms**:

*   The access provided to the LLM to specific documents that are relevant to our needs--this has tremendous potential for assisting roles in any industry. In the Games Localization industry, for instance, tasks such as legacy terminology checks, adherence to style guides, and ensuring character continuation, could all benefit from RAG improving accuracy and efficiency.
*   The ability to track answers back to their source, allowing the user to separate the factual document being retrieved from the content generated outside of the scope of our documents.

The use of a **powerful LLM such as gpt-3.5-turbo** was beneficial in this particular project for the following reasons:

*   The ability to sustain a two-way conversation, by prompting instructions to interact as a conversational partner, such as providing answers that follow certain style and asking questions relevant to the topic.
*   Increased ability to understand and contextualize user inputs, providing more accurate and contextually relevant responses.

😨 An interesting observation about the last answer in the Output example below:
*   I did not prompt the LLM to assume Rubén Darío's persona, this fact is not explicit in the source documents, and it was never mentioned in the conversation--gpt-3.5-turbo inferred and assumed the character based on a global understanding of the book and associations learned during its pretraining (that's my guess, at least).

## Output example

Emoji coding:


⛄ `temperature = 0.7`

🥵 `temperature = 1`

✅ Correct answer, cross-checked with source document

❌ Incorrect answer, cross-checked with source document



Human.-	`Buen día, ¿me permite un minuto para hacerle unas preguntas?`

*   ⛄ gpt-3.5-turbo.-	`Buen día, estimado interlocutor. Por supuesto, estoy dispuesto a concederle un minuto de mi valioso tiempo para escuchar sus preguntas y brindarle las respuestas que esté en mi capacidad proporcionar. Adelante, estoy todo oídos.`


Human.-	`¿Qué reuniones literarias era las que más disfrutaba?`
*   ⛄ gpt-3.5-turbo.-	`Las reuniones literarias que más disfrutaba eran las que se llevaban a cabo en la casa de don Rafael Obligado.` ✅

Human.-	`¿Cuál era su relación con don Rafael y cómo describiría su personalidad?`
*   ⛄ gpt-3.5-turbo.-	`Mi relación con don Rafael era de amistad y admiración mutua. Don Rafael era un hombre de gran carácter y determinación. Era un líder político apasionado y comprometido con sus ideales. Su personalidad era enérgica y carismática, capaz de cautivar a las masas con su elocuencia y convicción. Era un hombre de principios firmes y no temía enfrentarse a los desafíos que se le presentaban. Su valentía y determinación lo llevaron a sellar su historia con su trágico final.` ❌

Human.-	`¿Qué representa la poesía para usted?`
*   🥵 gpt-3.5-turbo.-	`La poesía, para mí, es como el susurro del viento entre las hojas de los árboles, es la melodía de los pájaros al amanecer y el aroma embriagador de las rosas en plena floración. Es el lenguaje de los sentimientos profundos y de las emociones indescriptibles. Representa una ventana hacia lo inefable, una forma de expresar lo inexpresable. La poesía es un bálsamo para el alma, un refugio en medio del caos, un remanso de paz en el agitado mar de la vida. ¿Y para usted, qué representa la poesía?` ✅ (At least, the answer resembles the imagery present in the source document: `Poética, sí, para quien sepa comprender el vaho de arte que flota sobre ese inconmesurable océano de tierra, sobre todo en los crepúsculos vespertinos y en los amaneceres.`)
*   ⛄ gpt-3.5-turbo.-	`La poesía para mí representa una ventana hacia la belleza y la profundidad de las emociones humanas. Es un lenguaje que va más allá de las palabras cotidianas y nos permite explorar los misterios del alma. A través de la poesía, podemos capturar momentos efímeros y transmitirlos de manera eterna. Es un medio para expresar nuestras experiencias más íntimas y conectarnos con el mundo que nos rodea de una manera única y trascendental. ¿Cuál es tu poema favorito y por qué?` ❌

Human.-	`Adoro 'Los motivos del lobo', lo aprendí para recitarlo en la escuela.`
*   ⛄ gpt-3.5-turbo.-	`Me alegra saber que también te gusta "Los motivos del lobo". Es una obra maravillosa de José Emilio Pacheco. Su poesía siempre ha tenido un impacto profundo en los lectores y es muy popular en las escuelas. ¿Te gustaría compartir algún otro poema que hayas aprendido para recitar en la escuela?` ❌ 😑
*   🥵 gpt-3.5-turbo.-	`Qué maravilloso que hayas aprendido "Los motivos del lobo" para recitarlo en la escuela. Es uno de los poemas más icónicos de mi obra y me alegra saber que lo disfrutas. ¿Hay algún otro poema que te gustaría aprender o recitar?` ✅ 😵
